In [1]:
%run my_import

#uri = socket.gethostname()+".local:5555"
uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
#uri = "com-exp.local:5555"
#uri = "advantech.local:5555"
io = ZmsgIO(uri)
io.debug = False
scan_ids = []


In [20]:
reply = io.doit(master_cmd_get_slave_descr)
yaml_msg = yaml.safe_load(reply['msg'])
scan_ids = yaml_msg.keys()
ids=list(scan_ids)
ids

[696]

In [21]:
reply = io.doit(SdoInfo(u'SDO_NAME').set_bid(ids[0]))
yaml_msg = yaml.safe_load(reply['msg'])
sdo_names = yaml_msg
sdo_names

['force_x',
 'force_y',
 'force_z',
 'torque_x',
 'torque_y',
 'torque_z',
 'fault',
 'rtt',
 'ts',
 'board_id',
 'analog_sample_freq_HZ',
 'dac_tune_bitflag',
 'module_params',
 'dac_tune_debouce',
 'fw_ver',
 'ctrl_status_cmd',
 'ctrl_status_cmd_ack',
 'flash_params_cmd',
 'flash_params_cmd_ack',
 'fault_ack',
 'temperature',
 'adc_idx',
 'adc_conv',
 'adc_raw',
 'dac_offset']

In [22]:
reply = io.doit(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(ids[0]))
yaml_msg = yaml.safe_load(reply['msg'])
yaml_msg

{'adc_conv': 0.0,
 'adc_idx': 0,
 'adc_raw': 0,
 'analog_sample_freq_HZ': 10000,
 'board_id': 696,
 'ctrl_status_cmd': 0,
 'ctrl_status_cmd_ack': 0,
 'dac_offset': 0,
 'dac_tune_bitflag': 0,
 'dac_tune_debouce': 5,
 'fault': 1,
 'fault_ack': 0,
 'flash_params_cmd': 0,
 'flash_params_cmd_ack': 0,
 'force_x': 0.90097,
 'force_y': 1.85745,
 'force_z': 1.178977,
 'fw_ver': 20091716,
 'module_params': 0,
 'rtt': 39271,
 'temperature': 0.0,
 'torque_x': 1.419165,
 'torque_y': 0.0,
 'torque_z': 0.0,
 'ts': 39271}

In [45]:
io.doit(ctrl_cmd_dac_tune.set_bid(696))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [46]:
s=SdoCmd(rd_sdo=['fw_ver','analog_sample_freq_HZ'],wr_sdo={'adc_idx': 0}).set_bid(ids[0])
asdict(s)
io.doit(s)

{'type': 1,
 'cmd_type': 7,
 'msg': '{analog_sample_freq_HZ: 10000, fw_ver: 20091716}'}

In [25]:
io.doit(SdoCmd(rd_sdo=[],wr_sdo={'adc_idx': 0}).set_bid(ids[0]))
io.doit(ctrl_cmd_get_adc.set_bid(ids[0]))
io.doit(SdoCmd(rd_sdo=['adc_conv','adc_raw'],wr_sdo={}).set_bid(ids[0]))

{'type': 1, 'cmd_type': 7, 'msg': '{adc_conv: 0.900665, adc_raw: 5906}'}

In [26]:
import time, sys
adc_conv = list()
adc_raw = list()
for j in range(1):
    for i in range(4):
        io.doit(SdoCmd(rd_sdo=[],wr_sdo={'adc_idx': i}).set_bid(ids[0]))
        io.doit(ctrl_cmd_get_adc.set_bid(ids[0]))
        reply=io.doit(SdoCmd(rd_sdo=['adc_conv','adc_raw'],wr_sdo={}).set_bid(ids[0]))
        yaml_msg = yaml.safe_load(reply['msg'])
        adc_raw.append(yaml_msg['adc_raw'])
        adc_conv.append(yaml_msg['adc_conv'])
        print(reply['msg'])
        time.sleep(0.1)
        
[hex(x) for x in adc_raw]

{adc_conv: 0.901885, adc_raw: 5914}
{adc_conv: 1.857145, adc_raw: 12178}
{adc_conv: 1.177452, adc_raw: 7721}
{adc_conv: 1.416115, adc_raw: 9286}


['0x171a', '0x2f92', '0x1e29', '0x2446']

In [17]:
import time, sys
def tune_adc_chan(adc_idx) :
    dac_offset = 0x7FFF
    hister = 2
    tuned = False
    while ( not tuned ):
        io.doit(SdoCmd(rd_sdo=[],wr_sdo={'adc_idx': adc_idx, 'dac_offset':dac_offset}).set_bid(ids[0]))
        io.doit(ctrl_cmd_set_dac.set_bid(ids[0]))
        io.doit(ctrl_cmd_get_adc.set_bid(ids[0]))
        reply=io.doit(SdoCmd(rd_sdo=['adc_conv','adc_raw'],wr_sdo={}).set_bid(ids[0]))
        yaml_msg = yaml.safe_load(reply['msg'])
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        if ( adc_raw > 0x1FFF+hister ) :
            if ( dac_offset < 0xFFFF ) :
                dac_offset += 1
        elif ( adc_raw < 0x1FFF-hister ) :
            if ( dac_offset > 0 ) :
                dac_offset -= 1
        else : 
            tuned = True
        time.sleep(0.01)
        yield(adc_raw, adc_conv)


In [28]:
for ch in range(4) :
    print("--- {0} ---".format(ch))
    for adc_raw, adc_conv in tune_adc_chan(ch) :
        print("\r%f 0x%X"%(adc_conv, adc_raw), end="")
    print()

--- 0 ---
1.248822 0x1FFD
--- 1 ---
1.249432 0x2001
--- 2 ---
1.248822 0x1FFD
--- 3 ---
1.248822 0x1FFD


In [ ]:
tune_adc_chan(1)

In [ ]:
tune_adc_chan(2)

In [28]:
tune_adc_chan(3)

1.44021 0x24e4
1.436397 0x24cb
1.439752 0x24e1
1.428772 0x2499
1.423435 0x2476
1.426332 0x2489
1.414895 0x243e
1.41276 0x2430
1.41032 0x2420
1.405745 0x2402
1.394612 0x23b9
1.391105 0x23a2
1.394155 0x23b6
1.39812 0x23d0
1.393698 0x23b3
1.389275 0x2396
1.383785 0x2372
1.364875 0x22f6
1.369602 0x2315
1.3664 0x2300
1.362893 0x22e9
1.358775 0x22ce
1.353437 0x22ab
1.36091 0x22dc
1.339103 0x224d
1.333917 0x222b
1.335442 0x2235
1.344898 0x2273
1.324615 0x21ee
1.3359 0x2238
1.327665 0x2202
1.31821 0x21c4
1.310585 0x2192
1.309212 0x2189
1.315617 0x21b3
1.30479 0x216c
1.29991 0x214c
1.29625 0x2134
1.30052 0x2150
1.281915 0x20d6
1.285575 0x20ee
1.291065 0x2112
1.27856 0x20c0
1.269105 0x2082
1.26636 0x2070
1.276577 0x20b3
1.260565 0x204a
1.259345 0x2042
1.247908 0x1ff7
1.259802 0x2045
1.256142 0x202d
1.254465 0x2022
1.246382 0x1fed
1.243942 0x1fdd
1.257515 0x2036
1.25172 0x2010
1.247298 0x1ff3
1.252635 0x2016
1.24623 0x1fec
1.252635 0x2016
1.247298 0x1ff3
1.24562 0x1fe8
1.257057 0x2033
1.253245 0x